In [2]:
import pandas as pd
regular_season_game_data_file_paths = ['C:\\Users\\don.ma\\code\\personal\\NBA-Data-2010-2024-main\\regular_season_box_scores_2010_2024_part_1.csv','C:\\Users\\don.ma\\code\\personal\\NBA-Data-2010-2024-main\\regular_season_box_scores_2010_2024_part_2.csv','C:\\Users\\don.ma\\code\\personal\\NBA-Data-2010-2024-main\\regular_season_box_scores_2010_2024_part_3.csv']
playoff_game_data_file_path = 'C:\\Users\\don.ma\\code\\personal\\NBA-Data-2010-2024-main\\play_off_box_scores_2010_2024.csv'
regular_season_team_data_file_path= 'C:\\Users\\don.ma\\code\\personal\\NBA-Data-2010-2024-main\\regular_season_totals_2010_2024.csv'
playoff_team_data_file_path = 'C:\\Users\\don.ma\\code\\personal\\NBA-Data-2010-2024-main\\play_off_box_scores_2010_2024.csv'


regular_season_game_df = pd.DataFrame()
for file in regular_season_game_data_file_paths:
    dataset = pd.read_csv(file)
    regular_season_game_df = pd.concat([regular_season_game_df,dataset])

dataset.to_csv('.\\test.csv')

In [6]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.title("NBA Player Performance Tracker")
df = pd.read_csv('csv\\player_season_stats.csv')  # Your cleaned data
player = st.selectbox("Select Player", df['personName'].unique())
player_data = df[df['personName'] == player]
fig = px.line(player_data, x='season_year', y='points', title=f"{player} PPG Trend")
st.plotly_chart(fig)

2025-09-12 16:12:42.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 16:12:42.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [5]:
import pandas as pd

# Assuming combined_df is your concatenated DataFrame from the first question
# If not, load it: combined_df = pd.read_csv('combined_data.csv')

# Step 1: Preprocess data
# Convert 'minutes' from 'MM:SS' string to float minutes (if it's already float, skip this)
def minutes_to_float(min_str):
    if pd.isna(min_str) or min_str == '':
        return 0.0
    try:
        mins, secs = map(int, min_str.split(':'))
        return mins + secs / 60.0
    except:
        return 0.0

regular_season_game_df['minutes_float'] = regular_season_game_df['minutes'].apply(minutes_to_float)

# Filter out non-playing rows (e.g., DNP) if 'comment' indicates they didn't play
# Adjust based on your data; assuming empty comment means played
combined_df = regular_season_game_df[regular_season_game_df['comment'].isna() | (regular_season_game_df['comment'] == '')]

# Ensure numeric types for stats
stat_columns = [
    'fieldGoalsMade', 'fieldGoalsAttempted', 'threePointersMade', 'threePointersAttempted',
    'freeThrowsMade', 'freeThrowsAttempted', 'reboundsOffensive', 'reboundsDefensive',
    'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers', 'foulsPersonal',
    'points', 'plusMinusPoints', 'minutes_float'
]
combined_df[stat_columns] = combined_df[stat_columns].apply(pd.to_numeric, errors='coerce').fillna(0)

# Step 2: Group by personId and season_year, aggregate stats
player_season_df = combined_df.groupby(['personId', 'season_year']).agg({
    'personName': lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],  # Most common name
    'position': lambda x: '/'.join(sorted(x.dropna().unique())) if not x.dropna().empty else 'N/A',  # Unique positions separated by '/', or 'N/A'
    'gameId': 'nunique',  # Games played
    'minutes_float': 'sum',  # Total minutes
    'fieldGoalsMade': 'sum',
    'fieldGoalsAttempted': 'sum',
    'threePointersMade': 'sum',
    'threePointersAttempted': 'sum',
    'freeThrowsMade': 'sum',
    'freeThrowsAttempted': 'sum',
    'reboundsOffensive': 'sum',
    'reboundsDefensive': 'sum',
    'reboundsTotal': 'sum',
    'assists': 'sum',
    'steals': 'sum',
    'blocks': 'sum',
    'turnovers': 'sum',
    'foulsPersonal': 'sum',
    'points': 'sum',
    'plusMinusPoints': 'sum'
}).reset_index()

# Rename games played
player_season_df.rename(columns={'gameId': 'games_played'}, inplace=True)

# Step 3: Calculate derived stats (percentages and per-game averages)
player_season_df['fg_percentage'] = (
    player_season_df['fieldGoalsMade'] / player_season_df['fieldGoalsAttempted'].replace(0, 1)
).fillna(0)
player_season_df['three_pt_percentage'] = (
    player_season_df['threePointersMade'] / player_season_df['threePointersAttempted'].replace(0, 1)
).fillna(0)
player_season_df['ft_percentage'] = (
    player_season_df['freeThrowsMade'] / player_season_df['freeThrowsAttempted'].replace(0, 1)
).fillna(0)

# Per-game averages (e.g., PPG, RPG)
player_season_df['avg_minutes'] = player_season_df['minutes_float'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_points'] = player_season_df['points'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_rebounds'] = player_season_df['reboundsTotal'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_assists'] = player_season_df['assists'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_steals'] = player_season_df['steals'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_blocks'] = player_season_df['blocks'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_turnovers'] = player_season_df['turnovers'] / player_season_df['games_played'].replace(0, 1)
player_season_df['avg_fouls'] = player_season_df['foulsPersonal'] / player_season_df['games_played'].replace(0, 1)

# Drop intermediate columns if not needed (e.g., keep totals and avgs)
# player_season_df.drop(columns=['minutes_float'], inplace=True)  # Uncomment if desired

# Final player table columns: personId, personName, season_year, and stats
print(player_season_df.head())  # View sample

# Optional: Save to CSV
player_season_df.to_csv('player_season_stats.csv', index=False)

C:\Users\don.ma\AppData\Local\Temp\ipykernel_16656\360274510.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df[stat_columns] = combined_df[stat_columns].apply(pd.to_numeric, errors='coerce').fillna(0)


   personId season_year        personName position  games_played  \
0       255     2010-11        Grant Hill        F            80   
1       255     2011-12        Grant Hill        F            49   
2       255     2012-13        Grant Hill      N/A            29   
3       406     2010-11  Shaquille O'Neal        C            37   
4       436     2010-11      Juwan Howard      N/A            57   

   minutes_float  fieldGoalsMade  fieldGoalsAttempted  threePointersMade  \
0    2408.900000             399                  825                 45   
1    1377.716667             201                  451                 14   
2     436.683333              38                   98                  3   
3     751.600000             134                  201                  0   
4     591.550000              55                  125                  0   

   threePointersAttempted  ...  three_pt_percentage  ft_percentage  \
0                     114  ...             0.394737       0.8287